# Directions
In the cell label Set your files... set the relvant parameters to the values you want them to have. The names should explain what they do, but there are comments to help. Run all the cells in order. The last cell will produce a sdf file that shares the name of your screening sdf file with "\_AD" at the end (test.sdf becomes test_AD.sdf). It will save it the the current working directory unless you specify a save location by setting the save_location variable in the Set cell

For reference it took me about 12 minutes to get the 13,000 of 460,000 compound in the AD of a 130 compound training set. This notebook run slower for reasons that I really dont understand but am going to blame on jupyter doing something crazy on the backend so here the runtime roughly doubled to about 20 minutes 

I spent a unresonable amount of time trying to make sure I protect this script from out of memory errors (screening datasets are too large). So you should be able to run even 10 million+ datasets on this with even 16gb of ram. If it crashes with a memory error, try changing the "10000" in np.array_splits function to 100000 (add a zero). If that doesn't work then you need a smaller screeening set and should complain to me that my script has a fatal error and I need to fix it

_(note to me, James, or the poor soul that has to maintain this: any out of memery issues is likley to come from the sdf file being too big for the sdf reader in rdkit to read in all at once. Sadly, because rdkit is not very nice there is no way to read it in chunks so you will need to write a wrapper function to split that sdf file into chunks and read them in as indivudal files to save memory)_

#### TODOs:
I never got the damn multiprocessing to work yet and that could cute runtime down by a factor of 10 if I could get it to work. But there is some shit about PiCkEliNg that python doesn't like and I don't even known why it has to do that to me so like whatever but some day if this puppy runs too slow I'll spend more than 30 minutes and 3 beers trying to figure it out


In [5]:
from rdkit.Chem import PandasTools, AllChem
from scipy.spatial import KDTree
from scipy.spatial.distance import pdist
import multiprocessing as mp
from tqdm import tqdm
import numpy as np
import time
import os

# Set your files and settings here

In [6]:
training_data_file = "test_zoe.sdf"  # Fill in your file name here
screening_data_file = "Enamine_Hit_Locator_Library/Enamine_Hit_Locator_Library_HLL-460_460160cmpds_20220221.sdf"  # Fill in your file name here
save_location = os.getcwd()  # set your save location here

fingerprint_radius = 5
fingerprint_bits = 2048
cpu_count = mp.cpu_count() - 1

# Some helper functions

In [7]:
def nn_runner(training_tree, screening_rows, cutoff):
    screening_rows["fp"] = screening_rows["ROMol"].apply(AllChem.GetMorganFingerprintAsBitVect, 
                                                         args=(fingerprint_radius, fingerprint_bits))
    screening_fp = np.array([list(x) for x in screening_rows["fp"]])

    # these are a cool data structure to rapidly find nearest neighbors ask me about them if you want me to fill you in
    #  for the non-cs nerds out there:
    #     knime fails to utilize something this efficient, so while here we only need to look up the distance of each
    #     screening point only once overall, in knime it has to do one look up per training data point. so if I had
    #     10,000 datapoint in training and 500,000 in screening with Kdtree I only need 500,000 for knime I need
    #     10,000 * 500,000 or 5,000,000,000. Even if you could do 10,000 calculations a second that is still 6 days
    #     of compute time compared to 50 seconds for the KDtree version
    #  for the cs nerds out there:
    #     knime pairwise distance function is O(n**2) while kdtrees are O(n) on build and O(n) on query so O(n) overall
    d, i = training_tree.query(screening_fp, workers=-1)

    res = np.where(d < cutoff)

    return screening_rows.iloc[res]


def get_euclidean_threshold(training_fp, mode=1, num_sd=2):
    # so there a few ways to do this, take the mean and sd from only nearest neighbors (mode=1) or
    #  use all distance pairs (mode=2) both make me feel unconformable as distributions are not defined here
    if mode == 1:
        training_tree = KDTree(training_fp)
        d, i = training_tree.query(training_fp, k=[1, 2], workers=-1)
    else:
        d = pdist(training_fp)

    m = np.mean(d)
    sd = np.std(d)

    # return the cutoff which is the mean + 2 standard deviations
    return m + (num_sd * sd)


def get_apd_threshold(training_fp, z=0.5):
    # this method is defined in S. Zhang, et. al., J. Chem. Inf. Model., 46 (2006), pp. 1984–1995
    #  and in the knime node domain: similarity node from the enalos community
    d = pdist(training_fp)
    m = np.mean(d)
    d_prime = d[np.where(d < m)]
    m_prime = np.mean(d_prime)
    sd_prime = np.std(d_prime)

    # return the cutoff which is the mean + (Z * standard deviation) of lower half set
    return m_prime + (z * sd_prime)

# The code to run

In [8]:
t1 = time.time()
print("Loading data...")
t0 = time.time()
training_df = PandasTools.LoadSDF(training_data_file)
print("loaded training:", str(time.time() - t0), "sec")
t0 = time.time()
screening_df = PandasTools.LoadSDF(screening_data_file)
print("loaded screening", str(time.time() - t0), "sec")
print("Done")

print("Processing training dataset...")
training_df["fp"] = training_df["ROMol"].apply(AllChem.GetMorganFingerprintAsBitVect, args=(fingerprint_radius, fingerprint_bits))
train_fp = np.array([list(x) for x in training_df["fp"]])
threshold = get_euclidean_threshold(train_fp)
print("Done")

print("Beginning AD evaluation...")


train_tree = KDTree(train_fp)
# NOTE: The 10000 seen in the progress bar is not the number of molecules to use, but the number of chucks to process
results = [nn_runner(training_tree=train_tree, screening_rows=x, cutoff=threshold)
           for x in tqdm(np.array_split(screening_df, 10000))]
print("Done")

new_file_name = screening_data_file.split("/")[-1].split(".")[0] + "_AD.sdf"
sav_loc = os.path.join(save_location, new_file_name)
print(f"Saving results to {sav_loc}")
df = pd.concat(results, ignore_index=True, axis=0)
PandasTools.WriteSDF(df, sav_loc, properties=list(df.columns))
print("Done")
print(f"Overall time: {time.time() - t1} sec")

"""
# pythons ability to multiprocess is bad
print("Beginning AD evaluation...")
with joblib.parallel_backend('multiprocessing'):
    Parallel(n_jobs=cpu_count)(delayed(partial(nn_runner, training_fp=train_fp.copy(), cutoff=threshold))(x)
                               for x in tqdm(np.array_split(screening_df, 50)))
print("Done")
"""

Loading data...
loaded training: 0.2902233600616455 sec
loaded screening 261.14604449272156 sec
Done
Processing training dataset...
Done
Beginning AD evaluation...


100%|██████████| 10000/10000 [18:16<00:00,  9.12it/s] 


Done


NameError: name 'save_loc' is not defined